## Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gc
import os, sys
import seaborn as sns
from tqdm import tqdm

from google.colab import files
import cv2

from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.layers import Conv2D,Flatten,MaxPool2D,BatchNormalization,GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121 as DenseNet
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.models import load_model

sns.set_style('darkgrid')

## Link data

In [ ]:
JSRT_dataset = '/content/drive/MyDrive/COURSES/CS431/Lung segmentation/Dataset/JSRT/preprocessed/'
nodule = 'Nodule/'
healthy ='Non-nodule/'

In [ ]:
df = {'file_name': [], 'label': []}
df = pd.DataFrame(df)
for filename in os.listdir(JSRT_dataset + nodule):
    new_row = {'file_name': nodule+filename, 'label': 'nodule'}
    df = df.append(new_row, ignore_index=True)

for filename in os.listdir(JSRT_dataset+healthy):
    new_row = {'file_name': healthy+filename, 'label': 'healthy'}
    df = df.append(new_row, ignore_index=True)
df

,file_name,label
0,Nodule/JPCLN072.png,nodule
1,Nodule/JPCLN066.png,nodule
2,Nodule/JPCLN067.png,nodule
3,Nodule/JPCLN098.png,nodule
4,Nodule/JPCLN107.png,nodule
...,...,...
188,Non-nodule/JPCNN033.png,healthy
189,Non-nodule/JPCNN027.png,healthy
190,Non-nodule/JPCLN094.png,healthy
191,Non-nodule/JPCNN026.png,healthy


## Split train and test


In [ ]:
# 90% train, 10% test
train_df, test_df = train_test_split(df, test_size=0.1, random_state = 42)

In [ ]:
X_df = train_df.drop(columns=['label'])
y_df = train_df['label']

## Image Data Generator

In [ ]:
batch_size = 32
img_height = 480
img_width = 480
target_size = (img_height, img_width)

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=10,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    rescale=1.,
                                    zoom_range=0.2,
                                    fill_mode='nearest',
                                    cval=0)

test_datagen = ImageDataGenerator(rescale=1.)

## Create model

In [ ]:
base_model = DenseNet(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))

In [ ]:
len(base_model.layers)

427

In [ ]:
for layer in base_model.layers[:427]:
    layer.trainable = False

In [ ]:
# for i, layer in enumerate(base_model.layers):
#     print(i, layer.name, "-", layer.trainable)

In [ ]:
model = tf.keras.Sequential([
    base_model,
    Flatten(),
    BatchNormalization(),
    Dense(128, activation='elu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='elu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(1, activation='sigmoid'),
])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=5e-4), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 15, 15, 1024)      7037504   
_________________________________________________________________
flatten (Flatten)            (None, 230400)            0         
_________________________________________________________________
batch_normalization (BatchNo (None, 230400)            921600    
_________________________________________________________________
dense (Dense)                (None, 128)               29491328  
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8

## Callbacks


In [ ]:
patience = 3
stop_patience = 5
factor = 0.1
callbacks = [
    ModelCheckpoint("/content/drive/MyDrive/COURSES/CS431/Lung segmentation/model_checkpoints/densenet.h5", save_weights_only=True, save_best_only=True, verbose = 1),
    EarlyStopping(patience=stop_patience, monitor='val_loss', verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=factor, patience=patience, min_lr=1e-6, verbose=1)
]

## Build model

In [ ]:
def create_model():
    base_model = DenseNet(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))
    for layer in base_model.layers[:427]:
        layer.trainable = False
    model = tf.keras.Sequential([
        base_model,
        Flatten(),
        BatchNormalization(),
        Dense(128, activation='elu'),
        Dropout(0.5),
        BatchNormalization(),
        Dense(64, activation='elu'),
        Dropout(0.5),
        BatchNormalization(),
        Dense(1, activation='sigmoid'),
    ])
    model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=5e-4), metrics=['accuracy'])
    return model

## K-fold and fit

In [ ]:
epochs = 30
KF_model = KerasClassifier(build_fn=model, epochs=epochs, batch_size=5, callbacks=callbacks, verbose=1)

In [ ]:
cv = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)

## Test model

In [ ]:
TRAIN_PATH = JSRT_dataset
j = 0
data_kfold = pd.DataFrame()

In [ ]:
for train_idx, val_idx in list(cv.split(X_df, y_df)):
    x_train_df = df.iloc[train_idx]
    x_valid_df = df.iloc[val_idx]

    j += 1

    training_set = train_datagen.flow_from_dataframe(dataframe=x_train_df, directory=TRAIN_PATH,

                                                     x_col="file_name", y_col="label",

                                                     class_mode="binary",

                                                     target_size=(img_height, img_width), batch_size=batch_size)

    validation_set = test_datagen.flow_from_dataframe(dataframe=x_valid_df, directory=TRAIN_PATH,

                                                            x_col="file_name", y_col='label',

                                                            class_mode="binary",

                                                            target_size=(img_height, img_width), batch_size=batch_size)

    model = create_model()

    history = model.fit_generator(training_set,
                                validation_data=validation_set,
                                epochs=epochs,
                                callbacks=callbacks,
                                verbose=1)

    test_generator = ImageDataGenerator(rescale=1.)

    test_set = test_generator.flow_from_dataframe(dataframe=test_df, directory=TRAIN_PATH,

                                                  x_col="file_name", y_col='label',

                                                  class_mode='binary',

                                                  target_size=(img_height, img_width))



    best_model = model
    best_model.evaluate(test_set)

    # predicted_class_indices = np.argmax(pred, axis=1)

    # data_kfold[j] = predicted_class_indices

    # gc.collect()

Found 155 validated image filenames belonging to 2 classes.
Found 18 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
5/5 [==============================] - 44s 6s/step - loss: 1.0289 - accuracy: 0.3742 - val_loss: 1.0919 - val_accuracy: 0.5556

Epoch 00001: val_loss improved from inf to 1.09195, saving model to /content/drive/MyDrive/COURSES/CS431/Lung segmentation/model_checkpoints/densenet.h5
Epoch 2/30
5/5 [==============================] - 22s 4s/step - loss: 0.7973 - accuracy: 0.5742 - val_loss: 0.8826 - val_accuracy: 0.6667

Epoch 00002: val_loss improved from 1.09195 to 0.88264, saving model to /content/drive/MyDrive/COURSES/CS431/Lung segmentation/model_checkpoints/densenet.h5
Epoch 3/30
5/5 [==============================] - 22s 4s/step - loss: 0.8979 - accuracy: 0.5290 - val_loss: 0.9169 - val_accuracy: 0.6667

Epoch 00003: val_loss did not improve from 0.88264
Epoch 4/30
5/5 [==============================] - 22s 4s/step - loss: 0.8497 - accuracy: 0.5097 - val_loss: 0.8561 - val_accuracy: 0.6111

Epoch 00004: val_loss improved from 0.88264 to 0.85614, saving model to /content/dr

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
5/5 [==============================] - 32s 5s/step - loss: 0.8205 - accuracy: 0.5548 - val_loss: 1.4880 - val_accuracy: 0.6111

Epoch 00001: val_loss did not improve from 0.85614
Epoch 2/30
5/5 [==============================] - 21s 4s/step - loss: 0.7540 - accuracy: 0.5677 - val_loss: 1.1047 - val_accuracy: 0.6111

Epoch 00002: val_loss did not improve from 0.85614
Epoch 3/30
5/5 [==============================] - 22s 4s/step - loss: 0.7122 - accuracy: 0.6258 - val_loss: 1.0221 - val_accuracy: 0.4444

Epoch 00003: val_loss did not improve from 0.85614
Epoch 4/30
5/5 [==============================] - 21s 4s/step - loss: 0.6729 - accuracy: 0.6000 - val_loss: 1.1268 - val_accuracy: 0.3333

Epoch 00004: val_loss did not improve from 0.85614
Epoch 5/30
5/5 [==============================] - 22s 4s/step - loss: 0.6934 - accuracy: 0.6581 - val_loss: 0.9562 - val_accuracy: 0.4444

Epoch 00005: val_loss did not improve from 0.85614
Epoch 6/30
5/5 [==============================] - 

...

Epoch 1/30
5/5 [==============================] - 33s 5s/step - loss: 0.8902 - accuracy: 0.4744 - val_loss: 0.7102 - val_accuracy: 0.7647

Epoch 00001: val_loss did not improve from 0.50495
Epoch 2/30
5/5 [==============================] - 21s 4s/step - loss: 1.0017 - accuracy: 0.5000 - val_loss: 0.8002 - val_accuracy: 0.6471

Epoch 00002: val_loss did not improve from 0.50495
Epoch 3/30
5/5 [==============================] - 21s 4s/step - loss: 0.8104 - accuracy: 0.5769 - val_loss: 0.8016 - val_accuracy: 0.7059

Epoch 00003: val_loss did not improve from 0.50495
Epoch 4/30
5/5 [==============================] - 22s 4s/step - loss: 0.8431 - accuracy: 0.5256 - val_loss: 0.7703 - val_accuracy: 0.7059

Epoch 00004: val_loss did not improve from 0.50495

Epoch 00004: ReduceLROnPlateau reducing learning rate to 5.0000002374872565e-05.
Epoch 5/30
5/5 [==============================] - 21s 4s/step - loss: 0.7407 - accuracy: 0.6282 - val_loss: 0.7341 - val_accuracy: 0.7059

Epoch 00005: val_lo

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
5/5 [==============================] - 32s 5s/step - loss: 0.8108 - accuracy: 0.5641 - val_loss: 1.1053 - val_accuracy: 0.5882

Epoch 00001: val_loss did not improve from 0.50495
Epoch 2/30
5/5 [==============================] - 21s 4s/step - loss: 0.8771 - accuracy: 0.4744 - val_loss: 0.8574 - val_accuracy: 0.6471

Epoch 00002: val_loss did not improve from 0.50495
Epoch 3/30
5/5 [==============================] - 21s 4s/step - loss: 0.7299 - accuracy: 0.5705 - val_loss: 0.7408 - val_accuracy: 0.7647

Epoch 00003: val_loss did not improve from 0.50495
Epoch 4/30
5/5 [==============================] - 21s 4s/step - loss: 0.7720 - accuracy: 0.5641 - val_loss: 0.7282 - val_accuracy: 0.7647

Epoch 00004: val_loss did not improve from 0.50495
Epoch 5/30
5/5 [==============================] - 21s 4s/step - loss: 0.7159 - accuracy: 0.6282 - val_loss: 0.7796 - val_accuracy: 0.5882

Epoch 00005: val_loss did not improve from 0.50495
Epoch 6/30
5/5 [==============================] - 

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
5/5 [==============================] - 34s 5s/step - loss: 0.8163 - accuracy: 0.5513 - val_loss: 1.1834 - val_accuracy: 0.4706

Epoch 00001: val_loss did not improve from 0.50495
Epoch 2/30
5/5 [==============================] - 22s 4s/step - loss: 0.7448 - accuracy: 0.6154 - val_loss: 0.9944 - val_accuracy: 0.4706

Epoch 00002: val_loss did not improve from 0.50495
Epoch 3/30
5/5 [==============================] - 21s 4s/step - loss: 0.7075 - accuracy: 0.6026 - val_loss: 1.0867 - val_accuracy: 0.5294

Epoch 00003: val_loss did not improve from 0.50495
Epoch 4/30
5/5 [==============================] - 21s 4s/step - loss: 0.7476 - accuracy: 0.5833 - val_loss: 0.8657 - val_accuracy: 0.5882

Epoch 00004: val_loss did not improve from 0.50495
Epoch 5/30
5/5 [==============================] - 21s 4s/step - loss: 0.7575 - accuracy: 0.5705 - val_loss: 0.8318 - val_accuracy: 0.5882

Epoch 00005: val_loss did not improve from 0.50495
Epoch 6/30
5/5 [==============================] - 